In [21]:
import pandas as pd
from sklearn.linear_model import Ridge

# Load your data
df = pd.read_csv("nfl_ra_epa_over_replacement_train.csv")

# Columns to keep
features = [
    'pass_share', 'completion_percentage', 'yards_per_completion',
    'tds_per_completion', 'rush_share', 'yards_per_rush', 'tds_per_rush', 'epa_per_snap',
    'reception_share', 'yards_per_reception', 'tds_per_reception'
]
target = 'ra_epa_over_replacement'
weight = 'total_games'

# Drop irrelevant columns
df = df.drop(columns=['gsis_id', 'player_display_name'], errors='ignore')

# Prepare output container
results = {}

# Loop through position groups
for position, group in df.groupby('position_group'):
    sample_weights = group[weight]
    X = group[features]
    y = group[target]
    print(sample_weights.dtype)

    # Drop rows with NaNs
    X = X.dropna()
    y = y.loc[X.index]
    sample_weights = sample_weights.loc[X.index]

    # Skip if no rows remain
    if len(X) == 0:
        continue

    # Fit model
    model = Ridge(alpha=1).fit(X, y,sample_weight=sample_weights)
    
    # Store coefficients
    coef = pd.Series(model.coef_, index=features)
    coef['intercept'] = model.intercept_
    
    results[position] = coef


# Create output DataFrame
coef_df = pd.DataFrame(results)
coef_df = coef_df.T
coef_df = coef_df.reset_index()
coef_df = coef_df.rename(columns={'index': 'position_group'})

# Save to CSV
coef_df.to_csv("nfl_ra_epa_coefficients.csv")

# Optional preview
print(coef_df)


int64
int64
int64
int64
int64
int64
int64
int64
int64
  position_group  pass_share  completion_percentage  yards_per_completion  \
0             QB    0.872746               0.526206              0.220211   
1             RB   -0.264810              -0.000082             -0.000106   
2             TE   -0.175420              -0.000008              0.000046   
3             WR   -0.430023               0.000290              0.001195   

   tds_per_completion  rush_share  yards_per_rush  tds_per_rush  epa_per_snap  \
0        1.075354e-01   -0.338550        0.343590      0.045234      0.352487   
1        2.486375e-06    0.152490        0.213962      0.024162      0.002106   
2        3.815612e-07   -0.025146       -0.000053      0.000064      0.001997   
3        4.623618e-07   -0.074300        0.008580     -0.000041      0.009866   

   reception_share  yards_per_reception  tds_per_reception  intercept  
0        -0.006436            -0.120345           0.000022  -3.519968  
1         

/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: 

In [18]:
import pandas as pd
import re
from datetime import date, timedelta

# Read the game_logs.csv file
df = pd.read_csv('nfl_ra_epa_coefficients.csv')

# Get the column names and data types
column_names = df.columns.tolist()
column_types = df.dtypes.tolist()

# Map pandas data types to SQL data types
type_mapping = {
    'object': 'varchar(255)',
    'int64': 'int',
    'float64': 'double precision',
    'bool': 'boolean'
}

# Replace spaces with underscores in column names
column_names = [name.replace(' ', '_') for name in column_names]
column_names = [re.sub(r'^(\d)', r'_\1', name) for name in column_names]

# Generate the SQL code
table_name = 'nfl_ra_epa_coefficients'
create_table_sql = f"create table {table_name}\n("
for column_name, column_type in zip(column_names, column_types):
    sql_type = type_mapping.get(str(column_type), 'varchar(255)')
    create_table_sql += f"    {column_name} {sql_type},\n"
create_table_sql = create_table_sql.rstrip(",\n")
create_table_sql += "\n);"
copy_sql = f"copy {table_name}({', '.join(column_names)})\n    from '/Users/riley.gisseman/Downloads/nfl_ra_epa_coefficients.csv'\n    delimiter ','\n    csv header;"

# Print the SQL code
print(create_table_sql)
print(copy_sql)


create table nfl_ra_epa_coefficients
(    Unnamed:_0 int,
    position_group varchar(255),
    snap_share double precision,
    pass_share double precision,
    completion_percentage double precision,
    yards_per_completion double precision,
    tds_per_completion double precision,
    rush_share double precision,
    yards_per_rush double precision,
    tds_per_rush double precision,
    epa_per_snap double precision,
    target_share double precision,
    catch_percentage double precision,
    yards_per_reception double precision,
    tds_per_reception double precision,
    intercept double precision
);
copy nfl_ra_epa_coefficients(Unnamed:_0, position_group, snap_share, pass_share, completion_percentage, yards_per_completion, tds_per_completion, rush_share, yards_per_rush, tds_per_rush, epa_per_snap, target_share, catch_percentage, yards_per_reception, tds_per_reception, intercept)
    from '/Users/riley.gisseman/Downloads/nfl_ra_epa_coefficients.csv'
    delimiter ','
    csv hea